In [ ]:
# =============================================================================
# Alpha Strategy Code Using yfinance + Enhanced Signal Logic
# =============================================================================

# Install Required Libraries (uncomment if needed)
# !pip install pandas matplotlib statsmodels scikit-learn joblib numba openpyxl yfinance

import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Use a non-interactive backend for faster plotting
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import warnings
import datetime
import os
from joblib import Parallel, delayed
import multiprocessing
import yfinance as yf  # <-- yfinance for data

warnings.filterwarnings("ignore")

# --------------------------------------------------------------------------
# CONFIGURATION: CSV file for Tickers + Directory/Paths
# --------------------------------------------------------------------------
# CSV with "Symbol" and "Description" columns
CSV_FILE_PATH = '/workspaces/Stock-Market-Prediction/Data/STOXX600 Tickers.csv'

# Output paths
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
OUTPUT_FILE_PATH = f'/workspaces/Stock-Market-Prediction/Quantative Investment/QI Output/analysis_results_STOXX_{today_date}.xlsx'
PLOT_DIR = '/workspaces/Stock-Market-Prediction/Quantative Investment/QI Output/plots'
os.makedirs(PLOT_DIR, exist_ok=True)

# Time range for yfinance data
# Set END_DATE to tomorrow to include today's data, since yfinance's end date is exclusive
END_DATE = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
START_DATE = '1980-01-01'  # Adjust as needed


# --------------------------------------------------------------------------
# Function to load Tickers from CSV
# --------------------------------------------------------------------------
def load_tickers(csv_path):
    """
    Loads tickers and descriptions from a CSV file.

    Parameters:
    - csv_path (str): Path to the CSV file.

    Returns:
    - List[str]: List of ticker symbols.
    - Dict[str, str]: Dictionary mapping ticker symbols to their descriptions.
    """
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"The CSV file '{csv_path}' does not exist. Please check the path.")

    df = pd.read_csv(csv_path)

    required_columns = {'Symbol', 'Description'}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"CSV file must contain columns: {required_columns}")

    # Handle duplicates
    df = df.drop_duplicates(subset='Symbol')

    tickers = df['Symbol'].tolist()
    descriptions = dict(zip(df['Symbol'], df['Description']))

    print(f"Loaded {len(tickers)} tickers from '{csv_path}'.")
    return tickers, descriptions


# --------------------------------------------------------------------------
# Fetch Data via yfinance
# --------------------------------------------------------------------------
def fetch_data_with_yfinance(tickers, start_date='1980-01-01', end_date=None):
    """
    Fetches historical Adjusted Close (or 'Close' if auto_adjust=True) prices
    for all tickers using yfinance.

    Parameters:
    - tickers (List[str]): List of ticker symbols.
    - start_date (str): Start date in 'YYYY-MM-DD' format.
    - end_date (str): End date in 'YYYY-MM-DD' format (default: tomorrow's date).

    Returns:
    - pd.DataFrame: DataFrame (indexed by date) with columns = ticker symbols.
    """
    if end_date is None:
        end_date = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

    print(f"Fetching data from yfinance for {len(tickers)} ticker(s)...")
    data = yf.download(
        tickers=tickers,
        start=start_date,
        end=end_date,
        progress=False,
        group_by='ticker',
        auto_adjust=True
    )

    # If only one ticker, data won't be multi-index
    if len(tickers) == 1:
        adj_close_df = data['Close'].to_frame(name=tickers[0])
    else:
        # If multiple tickers, data is multi-index
        adj_close_df = pd.DataFrame()
        for tkr in tickers:
            if ('Close' in data[tkr]):
                adj_close_df[tkr] = data[tkr]['Close']

    # Drop rows where all tickers are NaN
    adj_close_df.dropna(how='all', inplace=True)

    return adj_close_df


# --------------------------------------------------------------------------
# Load Tickers and Descriptions
# --------------------------------------------------------------------------
try:
    TICKERS, TICKER_DESCRIPTIONS = load_tickers(CSV_FILE_PATH)
except Exception as e:
    print(f"Error loading tickers: {e}")
    TICKERS = []
    TICKER_DESCRIPTIONS = {}


# --------------------------------------------------------------------------
# Preprocessing
# --------------------------------------------------------------------------
def preprocess_ticker_data(series):
    """
    Preprocesses the data for a single ticker by forward-filling,
    ensuring a business-day frequency, and dropping NaN.
    """
    series = series.dropna()
    series = series.asfreq('B')
    series.fillna(method='ffill', inplace=True)
    if not pd.api.types.is_datetime64_any_dtype(series.index):
        series.index = pd.to_datetime(series.index)
    return series


def log_transform(series):
    """Apply natural log transform, avoiding log(0) errors."""
    series = series.replace(0, 1e-9)
    return np.log(series)


# --------------------------------------------------------------------------
# Decomposition
# --------------------------------------------------------------------------
def decompose_time_series(series, model='additive', period=252):
    """
    Decomposes the time series into trend, seasonal, and residual components.
    """
    print(f"Performing '{model}' decomposition (period={period})...")
    return seasonal_decompose(series, model=model, period=period, extrapolate_trend='freq')


def plot_decomposition(result, title='Time Series Decomposition', ticker='', save_dir=PLOT_DIR):
    """Plot the decomposition results."""
    plt.figure(figsize=(14, 10))
    plt.suptitle(f"{title} - {ticker}", fontsize=16)

    plt.subplot(411)
    plt.plot(result.observed, color='blue', label='Observed')
    plt.legend(loc='upper left')

    plt.subplot(412)
    plt.plot(result.trend, color='orange', label='Trend')
    plt.legend(loc='upper left')

    plt.subplot(413)
    plt.plot(result.seasonal, color='green', label='Seasonal')
    plt.legend(loc='upper left')

    plt.subplot(414)
    plt.plot(result.resid, color='red', label='Residual')
    plt.legend(loc='upper left')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(save_dir, f"{ticker}_decomposition.png"), dpi=150)
    plt.close()


# --------------------------------------------------------------------------
# Trend Slope Calculation (Vectorized Rolling Regression)
# --------------------------------------------------------------------------
def calculate_trend_slope_vectorized(trend_series, window=252):
    """
    Calculates rolling linear regression slope on the trend component
    using a vectorized approach.
    """
    y = trend_series.dropna().values
    n = len(y)
    if n < window:
        return pd.Series(dtype=float)

    try:
        from numpy.lib.stride_tricks import sliding_window_view
        rolling_windows = sliding_window_view(y, window_shape=window)
    except AttributeError:
        # fallback for older numpy versions
        shape = (n - window + 1, window)
        strides = (y.strides[0], y.strides[0])
        rolling_windows = np.lib.stride_tricks.as_strided(y, shape=shape, strides=strides)

    X = np.arange(window)
    X_mean = X.mean()
    X_var = ((X - X_mean) ** 2).sum()

    Y_mean = rolling_windows.mean(axis=1)
    cov = (rolling_windows - Y_mean[:, np.newaxis]) @ (X - X_mean)

    slope = cov / X_var
    idx = trend_series.dropna().index[window-1:]
    return pd.Series(data=slope, index=idx)


# --------------------------------------------------------------------------
# Residual Z-Score
# --------------------------------------------------------------------------
def calculate_residual_zscore(residual_series, window=252):
    """
    Calculates rolling Z-score of the residual component.
    """
    rolling_mean = residual_series.rolling(window=window, min_periods=window).mean()
    rolling_std = residual_series.rolling(window=window, min_periods=window).std()
    z_score = (residual_series - rolling_mean) / rolling_std
    return z_score


# --------------------------------------------------------------------------
# Trend Strength Logic
# --------------------------------------------------------------------------
def get_trend_strength(slope):
    """
    Determines a multiplier based on absolute slope.
    """
    abs_slope = abs(slope)
    if abs_slope < 0.01:
        return 1.0  # Weak trend
    elif 0.01 <= abs_slope < 0.05:
        return 1.5  # Moderate trend
    else:
        return 2.0  # Strong trend


# --------------------------------------------------------------------------
# Backtest Strategy with Variable Position Sizing
# --------------------------------------------------------------------------
def backtest_strategy(analysis_df, ticker='', description=''):
    """
    Enhanced backtest:
    - Buys incrementally when Z-score < -2
    - Sells incrementally when Z-score > 2
    - Position sizing is influenced by Z-score magnitude * trend strength.
    - max_position = 3.0 (example) to limit total position.

    Returns:
    - performance_metrics: dict
    - last_trade_signal_details: (signal_type, signal_date, signal_strength)
    """
    analysis_df['Position'] = 0.0
    last_trade_signal_type = 'N/A'
    last_signal_date = 'N/A'
    last_signal_strength = 'N/A'

    position = 0.0
    max_position = 3.0  # maximum position size
    for idx, row in analysis_df.iterrows():
        current_slope = row['Trend_Slope'] if not pd.isna(row['Trend_Slope']) else 0.0
        trend_strength = get_trend_strength(current_slope)

        z_score = row['Residual_Z_Score']
        signal_strength = 0.0
        signal_type = None

        if z_score < -2:
            # Potential Buy
            signal_strength = abs(z_score) * trend_strength
            # Cap at max position
            if position + signal_strength > max_position:
                signal_strength = max_position - position

            if signal_strength > 0:
                signal_type = 'Buy'
                position += signal_strength
                last_trade_signal_type = signal_type
                last_signal_date = idx.strftime('%Y-%m-%d')
                last_signal_strength = round(signal_strength, 2)
                print(f"[{ticker}] Buy on {last_signal_date} (Strength: {last_signal_strength})")

        elif z_score > 2:
            # Potential Sell
            signal_strength = z_score * trend_strength
            # Cap so we don't go negative
            if position - signal_strength < 0:
                signal_strength = position

            if signal_strength > 0:
                signal_type = 'Sell'
                position -= signal_strength
                last_trade_signal_type = signal_type
                last_signal_date = idx.strftime('%Y-%m-%d')
                last_signal_strength = round(signal_strength, 2)
                print(f"[{ticker}] Sell on {last_signal_date} (Strength: {last_signal_strength})")

        analysis_df.at[idx, 'Position'] = position

    # --------------------------------------------------------------------------
    # Modify Position_Shifted to apply signals on the same day
    # --------------------------------------------------------------------------
    # Remove the shift to apply the position on the same day
    analysis_df['Position_Shifted'] = analysis_df['Position']  # Changed from shift(1)

    # Compute returns
    analysis_df['Log_Return'] = analysis_df['Adj_Close'].diff()
    analysis_df['Strategy_Return'] = analysis_df['Position_Shifted'] * analysis_df['Log_Return']
    analysis_df['Strategy_Return'].fillna(0.0, inplace=True)

    # Cumulative returns
    analysis_df['Cumulative_Strategy_Return'] = analysis_df['Strategy_Return'].cumsum().apply(np.exp)
    analysis_df['Cumulative_Buy_and_Hold_Return'] = analysis_df['Log_Return'].cumsum().apply(np.exp)

    # Performance stats
    strategy_total_return = analysis_df['Cumulative_Strategy_Return'].iloc[-1] - 1
    buy_hold_total_return = analysis_df['Cumulative_Buy_and_Hold_Return'].iloc[-1] - 1
    strategy_years = (analysis_df.index[-1] - analysis_df.index[0]).days / 365.25

    if strategy_years <= 0:
        strategy_annual_return = 0.0
        buy_hold_annual_return = 0.0
    else:
        strategy_annual_return = (analysis_df['Cumulative_Strategy_Return'].iloc[-1])**(1/strategy_years) - 1
        buy_hold_annual_return = (analysis_df['Cumulative_Buy_and_Hold_Return'].iloc[-1])**(1/strategy_years) - 1

    strategy_volatility = analysis_df['Strategy_Return'].std() * np.sqrt(252)
    buy_hold_volatility = analysis_df['Log_Return'].std() * np.sqrt(252)

    strategy_sharpe = strategy_annual_return / strategy_volatility if strategy_volatility != 0 else 0.0
    buy_hold_sharpe = buy_hold_annual_return / buy_hold_volatility if buy_hold_volatility != 0 else 0.0

    # Drawdown
    analysis_df['Strategy_Cumulative'] = analysis_df['Strategy_Return'].cumsum().apply(np.exp)
    analysis_df['Strategy_Cumulative_Max'] = analysis_df['Strategy_Cumulative'].cummax()
    analysis_df['Strategy_Drawdown'] = analysis_df['Strategy_Cumulative'] / analysis_df['Strategy_Cumulative_Max'] - 1
    strategy_max_drawdown = analysis_df['Strategy_Drawdown'].min()

    analysis_df['Buy_Hold_Cumulative'] = analysis_df['Log_Return'].cumsum().apply(np.exp)
    analysis_df['Buy_Hold_Cumulative_Max'] = analysis_df['Buy_Hold_Cumulative'].cummax()
    analysis_df['Buy_Hold_Drawdown'] = analysis_df['Buy_Hold_Cumulative'] / analysis_df['Buy_Hold_Cumulative_Max'] - 1
    buy_hold_max_drawdown = analysis_df['Buy_Hold_Drawdown'].min()

    performance_metrics = {
        'Strategy Annual Return': strategy_annual_return,
        'Buy and Hold Annual Return': buy_hold_annual_return,
        'Strategy Sharpe': strategy_sharpe,
        'Buy and Hold Sharpe': buy_hold_sharpe,
        'Strategy Maximum Drawdown': strategy_max_drawdown,
        'Buy and Hold Maximum Drawdown': buy_hold_max_drawdown
    }
    return performance_metrics, (last_trade_signal_type, last_signal_date, last_signal_strength)


# --------------------------------------------------------------------------
# Analyze Momentum
# --------------------------------------------------------------------------
def analyze_momentum(decompose_result, original_series, ticker='', description=''):
    """
    Analyzes momentum based on the trend and residual components,
    plus backtests the strategy.

    Returns:
      - trend_status
      - residual_status
      - performance_metrics
      - last_trade_signal_details
    """
    trend = decompose_result.trend
    residual = decompose_result.resid

    # Trend slope
    slope_series = calculate_trend_slope_vectorized(trend, window=252)
    # Residual z-score
    z_score = calculate_residual_zscore(residual, window=252).rename('Residual_Z_Score')

    # Combine
    analysis_df = pd.concat([
        original_series.rename('Adj_Close'),
        trend.rename('Trend'),
        slope_series.rename('Trend_Slope'),
        residual.rename('Residual'),
        z_score
    ], axis=1)

    # Plot Trend Slope
    plt.figure(figsize=(14, 6))
    plt.plot(analysis_df.index, analysis_df['Trend_Slope'], color='purple', label='Trend Slope (1Yr Rolling)')
    plt.title(f'Trend Slope - {ticker}')
    plt.xlabel('Date')
    plt.ylabel('Slope')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, f"{ticker}_trend_slope.png"), dpi=150)
    plt.close()

    # Plot Residual Z-Score
    plt.figure(figsize=(14, 6))
    plt.plot(analysis_df.index, analysis_df['Residual_Z_Score'], color='brown', label='Residual Z-Score (1Yr Rolling)')
    plt.axhline(2, color='red', linestyle='--', label='±2')
    plt.axhline(-2, color='green', linestyle='--')
    plt.axhline(3, color='darkred', linestyle='--', label='±3')
    plt.axhline(-3, color='darkgreen', linestyle='--')
    plt.axhline(4, color='maroon', linestyle='--', label='±4')
    plt.axhline(-4, color='darkolivegreen', linestyle='--')
    plt.title(f'Residual Z-Score - {ticker}')
    plt.xlabel('Date')
    plt.ylabel('Z-Score')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, f"{ticker}_residual_zscore.png"), dpi=150)
    plt.close()

    # Interpretation
    trend_status = "Insufficient Data"
    residual_status = "Insufficient Data"
    if not slope_series.dropna().empty and not z_score.dropna().empty:
        latest_slope = slope_series.dropna().iloc[-1]
        latest_z = z_score.dropna().iloc[-1]

        # Trend Status
        if latest_slope > 0:
            trend_status = "Upward Momentum"
        elif latest_slope < 0:
            trend_status = "Downward Momentum"
        else:
            trend_status = "No Clear Momentum"

        # Residual Status with multi-level thresholds
        if latest_z > 4:
            residual_status = "Highly Overbought Condition"
        elif latest_z > 3:
            residual_status = "Moderately Overbought Condition"
        elif latest_z > 2:
            residual_status = "Slightly Overbought Condition"
        elif latest_z < -4:
            residual_status = "Highly Oversold Condition"
        elif latest_z < -3:
            residual_status = "Moderately Oversold Condition"
        elif latest_z < -2:
            residual_status = "Slightly Oversold Condition"
        else:
            residual_status = "Residuals Within Normal Range"

    # Backtest
    performance_metrics, last_trade_signal_details = backtest_strategy(analysis_df, ticker, description)

    return trend_status, residual_status, performance_metrics, last_trade_signal_details


# --------------------------------------------------------------------------
# Process a Single Ticker
# --------------------------------------------------------------------------
def process_ticker(ticker, description, adj_close_data):
    """
    Runs the entire pipeline for a single ticker:
      - Preprocess
      - Log transform
      - Decompose
      - Analyze momentum
      - Backtest
    """
    result = {
        'Ticker': ticker,
        'Description': description,
        'Trend Status': 'N/A',
        'Residual Status': 'N/A',
        'Strategy Annual Return': np.nan,
        'Buy and Hold Annual Return': np.nan,
        'Strategy Sharpe': np.nan,
        'Buy and Hold Sharpe': np.nan,
        'Strategy Maximum Drawdown': np.nan,
        'Buy and Hold Maximum Drawdown': np.nan,
        'Last Trade Signal': 'N/A',
        'Last Signal Date': 'N/A',
        'Last Signal Strength': 'N/A'
    }

    if ticker not in adj_close_data.columns:
        return result

    ticker_series = adj_close_data[ticker].dropna()
    if ticker_series.empty:
        return result

    # 1) Preprocess
    preprocessed_series = preprocess_ticker_data(ticker_series)
    # 2) Log transform
    log_series = log_transform(preprocessed_series)

    # 3) Decompose
    try:
        additive_result = decompose_time_series(log_series, model='additive', period=252)
    except Exception:
        return result  # not enough data or decomposition error

    # 4) Plot Decomposition
    plot_decomposition(
        additive_result,
        title='Additive Decomposition of Log-Transformed Data',
        ticker=ticker
    )

    # 5) Analyze Momentum & Backtest
    trend_status, residual_status, performance_metrics, last_trade_signal_details = \
        analyze_momentum(additive_result, log_series, ticker, description)

    # Update result
    result.update({
        'Trend Status': trend_status,
        'Residual Status': residual_status,
        'Strategy Annual Return': performance_metrics['Strategy Annual Return'],
        'Buy and Hold Annual Return': performance_metrics['Buy and Hold Annual Return'],
        'Strategy Sharpe': performance_metrics['Strategy Sharpe'],
        'Buy and Hold Sharpe': performance_metrics['Buy and Hold Sharpe'],
        'Strategy Maximum Drawdown': performance_metrics['Strategy Maximum Drawdown'],
        'Buy and Hold Maximum Drawdown': performance_metrics['Buy and Hold Maximum Drawdown']
    })

    last_trade_signal_type, last_signal_date, last_signal_strength = last_trade_signal_details
    if last_trade_signal_type in ['Buy', 'Sell']:
        result['Last Trade Signal'] = last_trade_signal_type
        result['Last Signal Date'] = last_signal_date
        result['Last Signal Strength'] = last_signal_strength

    return result


# --------------------------------------------------------------------------
# Main Execution
# --------------------------------------------------------------------------
def main():
    if not TICKERS:
        print("No tickers found. Exiting.")
        return

    # 1) Fetch data from yfinance
    print("\nFetching data from yfinance...")
    adj_close_data = fetch_data_with_yfinance(
        tickers=TICKERS,
        start_date=START_DATE,
        end_date=END_DATE
    )

    # Filter only those tickers that actually returned data
    common_tickers = [t for t in TICKERS if t in adj_close_data.columns]
    if not common_tickers:
        print("No matching tickers found in yfinance data. Exiting.")
        return

    adj_close_data = adj_close_data[common_tickers]

    # 2) Parallel processing
    num_cores = multiprocessing.cpu_count()
    print(f"Using {num_cores} CPU cores for parallel processing.")
    results = Parallel(n_jobs=num_cores)(
        delayed(process_ticker)(
            ticker,
            TICKER_DESCRIPTIONS.get(ticker, 'No Description Available'),
            adj_close_data
        )
        for ticker in common_tickers
    )

    # 3) Prepare and save results
    if results:
        results_df = pd.DataFrame(results)
        columns_order = [
            'Ticker',
            'Description',
            'Trend Status',
            'Residual Status',
            'Strategy Annual Return',
            'Buy and Hold Annual Return',
            'Strategy Sharpe',
            'Buy and Hold Sharpe',
            'Strategy Maximum Drawdown',
            'Buy and Hold Maximum Drawdown',
            'Last Trade Signal',
            'Last Signal Date',
            'Last Signal Strength'
        ]
        results_df = results_df[columns_order]
        results_df.to_excel(OUTPUT_FILE_PATH, index=False)
        print(f"\nAnalysis results saved to '{OUTPUT_FILE_PATH}'.")
    else:
        print("No results generated. Exiting.")

    print("\nAll tickers processed successfully.")


# Run Main
if __name__ == "__main__":
    main()


In [5]:
import pandas as pd
import numpy as np
import sys
import re
from datetime import datetime
import os
import gspread
from google.oauth2.service_account import Credentials
import time
import logging
from gspread.exceptions import APIError, WorksheetNotFound, SpreadsheetNotFound

# Configure logging
logging.basicConfig(
    filename='/workspaces/Stock-Market-Prediction/Quantative Investment/QI Output/script.log',  
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:%(message)s'
)

def extract_date_from_filename(filename):
    """
    Extracts a date in YYYY-MM-DD format from the given filename.
    If no date is found, returns None.
    """
    match = re.search(r'(\d{4}-\d{2}-\d{2})', filename)
    if match:
        return match.group(1)
    return None

def authenticate_google_sheets(credentials_path):
    """
    Authenticates and returns a gspread client.
    """
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]
    credentials = Credentials.from_service_account_file(
        credentials_path,
        scopes=SCOPES
    )
    client = gspread.authorize(credentials)
    return client

def update_google_sheet(client, sheet_name, dataframe, worksheet_title, max_retries=5):
    """
    Updates the specified worksheet in the given Google Sheet with the provided DataFrame using batch update.
    Implements exponential backoff for handling rate limits.

    This function always updates the worksheet defined by worksheet_title.
    If the worksheet does not exist, it creates it.
    """
    try:
        spreadsheet = client.open(sheet_name)
    except SpreadsheetNotFound:
        logging.error(f"The Google Sheet '{sheet_name}' was not found.")
        print(f"Error: The Google Sheet '{sheet_name}' was not found.")
        sys.exit(1)
    except Exception as e:
        logging.error(f"An error occurred while opening the Google Sheet: {e}")
        print(f"An error occurred while opening the Google Sheet: {e}")
        sys.exit(1)

    # Get the desired worksheet
    try:
        worksheet = spreadsheet.worksheet(worksheet_title)
    except WorksheetNotFound:
        try:
            # Create a new worksheet with default dimensions (adjust rows and cols as needed)
            worksheet = spreadsheet.add_worksheet(title=worksheet_title, rows="100", cols="20")
            logging.info(f"Worksheet '{worksheet_title}' created in Google Sheet '{sheet_name}'.")
            print(f"Worksheet '{worksheet_title}' created in Google Sheet '{sheet_name}'.")
        except Exception as e:
            logging.error(f"Failed to create worksheet '{worksheet_title}': {e}")
            print(f"Failed to create worksheet '{worksheet_title}': {e}")
            sys.exit(1)

    # Clear existing content in the target worksheet
    try:
        worksheet.clear()
        logging.info(f"Cleared existing content in the worksheet '{worksheet_title}'.")
    except Exception as e:
        logging.error(f"Failed to clear the worksheet: {e}")
        print(f"Failed to clear the worksheet: {e}")
        sys.exit(1)

    # Prepare data for insertion
    headers = list(dataframe.columns)
    data = dataframe.values.tolist()
    all_data = [headers] + data

    # Determine the range to update (starting at A1)
    cell_range = 'A1'

    attempt = 0
    while attempt < max_retries:
        try:
            worksheet.update(cell_range, all_data)
            logging.info(f"Successfully updated the worksheet '{worksheet_title}' in Google Sheet '{sheet_name}'.")
            print(f"Successfully updated the worksheet '{worksheet_title}' in Google Sheet '{sheet_name}'.")
            break
        except APIError as api_err:
            if api_err.response.status_code == 429:
                wait_time = 2 ** attempt
                logging.warning(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
                attempt += 1
            else:
                logging.error(f"An API error occurred: {api_err}")
                print(f"An API error occurred: {api_err}")
                sys.exit(1)
        except Exception as e:
            logging.error(f"An unexpected error occurred while updating the worksheet: {e}")
            print(f"An unexpected error occurred while updating the worksheet: {e}")
            sys.exit(1)
    else:
        logging.error("Max retries exceeded. Failed to update the worksheet.")
        print("Max retries exceeded. Failed to update the worksheet.")
        sys.exit(1)

def main():
    # Define file paths
    main_excel_path = '/workspaces/Stock-Market-Prediction/Quantative Investment/QI Output/analysis_results_STOXX_2025-05-18.xlsx'  # Path to your main Excel file
    isin_csv_path = '/workspaces/Stock-Market-Prediction/Data/STOXX Tickers with ISIN.csv'  # Path to your ISIN CSV file
    qi_folder_path = '/workspaces/Stock-Market-Prediction/Quantative Investment/QI Output'  # Path to the QI folder in Google Drive
    service_account_path = '/workspaces/Stock-Market-Prediction/Data/Google Sheets - webhook-442401-7a057b78bd8f.json'  # Path to your service account JSON file
    google_sheet_name = 'QI_S&P'  # Replace with your Google Sheet name

    # Extract date from main_excel_path
    date_str = extract_date_from_filename(main_excel_path)
    if date_str:
        logging.info(f"Extracted date from filename: {date_str}")
        print(f"Extracted date from filename: {date_str}")
    else:
        date_str = datetime.now().strftime('%Y-%m-%d')
        logging.info(f"No date found in filename. Using current date: {date_str}")
        print(f"No date found in filename. Using current date: {date_str}")

    # Create a new folder in the QI directory for the date
    new_folder_path = os.path.join(qi_folder_path, date_str)
    try:
        os.makedirs(new_folder_path, exist_ok=True)
        logging.info(f"Created or verified folder: {new_folder_path}")
        print(f"Created or verified folder: {new_folder_path}")
    except Exception as e:
        logging.error(f"Failed to create or verify folder '{new_folder_path}': {e}")
        print(f"Failed to create or verify folder '{new_folder_path}': {e}")
        sys.exit(1)

    # Construct the output CSV file path
    output_csv_path = os.path.join(new_folder_path, f"QI_S&P_{date_str}.csv")

    # Read the main Excel file
    try:
        main_df = pd.read_excel(main_excel_path, engine='openpyxl')
        logging.info(f"Successfully read the main Excel file: '{main_excel_path}'")
        print(f"Successfully read the main Excel file: '{main_excel_path}'")
    except FileNotFoundError:
        logging.error(f"The file '{main_excel_path}' was not found.")
        print(f"Error: The file '{main_excel_path}' was not found.")
        sys.exit(1)
    except Exception as e:
        logging.error(f"Error reading '{main_excel_path}': {e}")
        print(f"Error reading '{main_excel_path}': {e}")
        sys.exit(1)

    # Read the ISIN CSV file
    try:
        isin_df = pd.read_csv(isin_csv_path)
        logging.info(f"Successfully read the ISIN CSV file: '{isin_csv_path}'")
        print(f"Successfully read the ISIN CSV file: '{isin_csv_path}'")
    except FileNotFoundError:
        logging.error(f"The file '{isin_csv_path}' was not found.")
        print(f"Error: The file '{isin_csv_path}' was not found.")
        sys.exit(1)
    except Exception as e:
        logging.error(f"Error reading '{isin_csv_path}': {e}")
        print(f"Error reading '{isin_csv_path}': {e}")
        sys.exit(1)

    # Ensure consistent case for merging
    main_df['Ticker'] = main_df['Ticker'].str.upper()
    isin_df['Symbol'] = isin_df['Symbol'].str.upper()

    # Remove duplicate symbols in ISIN file if any
    if isin_df['Symbol'].duplicated().any():
        logging.warning("Duplicate symbols found in ISIN file. Keeping the first occurrence.")
        print("Warning: Duplicate symbols found in ISIN file. Keeping the first occurrence.")
        isin_df.drop_duplicates(subset=['Symbol'], keep='first', inplace=True)

    # Merge the DataFrames on Ticker (from main_df) and Symbol (from isin_df)
    try:
        merged_df = pd.merge(
            main_df,
            isin_df[['Symbol', 'International Securities Identification Number']],
            left_on='Ticker',
            right_on='Symbol',
            how='left'
        )
        logging.info("Successfully merged DataFrames.")
        print("Successfully merged DataFrames.")
    except Exception as e:
        logging.error(f"Error during merging DataFrames: {e}")
        print(f"Error during merging DataFrames: {e}")
        sys.exit(1)

    # Drop the redundant 'Symbol' column and rename the ISIN column for clarity
    merged_df.drop('Symbol', axis=1, inplace=True)
    merged_df.rename(columns={'International Securities Identification Number': 'ISIN'}, inplace=True)

    # Check for any missing ISINs
    missing_isin = merged_df['ISIN'].isnull().sum()
    if missing_isin > 0:
        logging.warning(f"{missing_isin} ticker(s) did not have a matching ISIN.")
        print(f"Warning: {missing_isin} ticker(s) did not have a matching ISIN.")

    # Sort the DataFrame by 'Last Signal Date' from newest to oldest
    date_column = 'Last Signal Date'
    if date_column in merged_df.columns:
        try:
            merged_df[date_column] = pd.to_datetime(merged_df[date_column], errors='coerce')
            merged_df.sort_values(by=date_column, ascending=False, inplace=True)
            logging.info(f"Successfully sorted the DataFrame by '{date_column}' in descending order.")
            print(f"Successfully sorted the DataFrame by '{date_column}' in descending order.")
        except Exception as e:
            logging.error(f"Error sorting the DataFrame by '{date_column}': {e}")
            print(f"Error sorting the DataFrame by '{date_column}': {e}")
            sys.exit(1)
    else:
        logging.warning(f"The column '{date_column}' does not exist in the DataFrame.")
        print(f"Warning: The column '{date_column}' does not exist in the DataFrame.")

    # Convert datetime columns to string for JSON serialization
    for column in merged_df.columns:
        if pd.api.types.is_datetime64_any_dtype(merged_df[column]):
            try:
                merged_df[column] = merged_df[column].dt.strftime('%Y-%m-%d %H:%M:%S')
                logging.info(f"Converted datetime column '{column}' to string format.")
                print(f"Converted datetime column '{column}' to string format.")
            except Exception as e:
                logging.error(f"Error converting column '{column}' to string: {e}")
                print(f"Error converting column '{column}' to string: {e}")
                sys.exit(1)

    # Replace infinite values with NaN and fill NaNs with an empty string
    merged_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    merged_df.fillna("", inplace=True)

    # Save the merged DataFrame to CSV
    try:
        merged_df.to_csv(output_csv_path, index=False)
        logging.info(f"Successfully saved the merged data to '{output_csv_path}'")
        print(f"Successfully saved the merged data to '{output_csv_path}'")
    except Exception as e:
        logging.error(f"Error saving to '{output_csv_path}': {e}")
        print(f"Error saving to '{output_csv_path}': {e}")
        sys.exit(1)

    # Authenticate and update only the "QI_STOXX600" worksheet in the Google Sheet
    try:
        client = authenticate_google_sheets(service_account_path)
        update_google_sheet(client, google_sheet_name, merged_df, worksheet_title="QI_STOXX600")
    except Exception as e:
        logging.error(f"An error occurred while updating the Google Sheet: {e}")
        print(f"An error occurred while updating the Google Sheet: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()


Extracted date from filename: 2025-05-18
Created or verified folder: /workspaces/Stock-Market-Prediction/Quantative Investment/QI Output/2025-05-18
Successfully read the main Excel file: '/workspaces/Stock-Market-Prediction/Quantative Investment/QI Output/analysis_results_STOXX_2025-05-18.xlsx'
Successfully read the ISIN CSV file: '/workspaces/Stock-Market-Prediction/Data/STOXX Tickers with ISIN.csv'
Successfully merged DataFrames.
Successfully sorted the DataFrame by 'Last Signal Date' in descending order.
Converted datetime column 'Last Signal Date' to string format.
Successfully saved the merged data to '/workspaces/Stock-Market-Prediction/Quantative Investment/QI Output/2025-05-18/QI_S&P_2025-05-18.csv'
Successfully updated the worksheet 'QI_STOXX600' in Google Sheet 'QI_S&P'.
